In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets, metrics
from sklearn.preprocessing import StandardScaler, normalize, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from copy import deepcopy

import time
from datetime import datetime

In [2]:
RANDOM_SEED      = 42


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")



In [3]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [4]:
xor_inputs[:5]

[[0.46017335909155244, 0.7885475077946231],
 [0.8015664915968379, -0.15680671292793424],
 [0.6552814993490459, -0.054154277627632474],
 [0.07733256366163754, -0.5455376330956947],
 [-0.8279523340150154, -0.7617755115787825]]

In [5]:
xor_outputs[:5]

[[0.0], [1.0], [1.0], [1.0], [0.0]]

# Set up run


In [6]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

In [7]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

In [8]:
def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

In [9]:
config = base_config
saveLocation = './'
maxNGenerations = 3
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 5)

g = p.best_genome


The function - generationStart - has just started at 1614481310.349806

 ****** Running generation 0 ****** 

The function - generationStart - took 0.0002789497375488281 seconds to complete
The function - pre_backprop - has just started at 1614481310.3501172
The function - pre_backprop - took 7.987022399902344e-05 seconds to complete
The function - backprop - has just started at 1614481310.350234
about to start backprop with 5 epochs
mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6689, grad_fn=<DivBackward0>)
The function - backprop - took 2.4360978603363037 seconds to complete
The function - post_backprop - has just started at 1614481312.786432
The function - post_backprop - took 2.5033950805664062e-05 seconds to complete
The function - evaluate fitness - has just started at 1614481312.786518
{1: <neat.genome.DefaultGenome object at 0x7fbcd0ef8d60>, 2: <neat.genome.DefaultGenome object at 0x7fbcd0ef8d90>, 3: <neat.genome.DefaultGenome ob

In [10]:
print(g)

Key: 9
Fitness: 1.0968289375305176
Nodes:
	0 DefaultNodeGene(key=0, bias=0.2840055525302887, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.6365209221839905, enabled=False)
	DefaultConnectionGene(key=(-1, 0), weight=-0.11565177142620087, enabled=True)


In [11]:
h = deepcopy(g)

In [12]:
print(h)

Key: 9
Fitness: 1.0968289375305176
Nodes:
	0 DefaultNodeGene(key=0, bias=0.2840055525302887, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.6365209221839905, enabled=False)
	DefaultConnectionGene(key=(-1, 0), weight=-0.11565177142620087, enabled=True)


In [13]:
h.mutate_add_node(p.config.genome_config)
h.mutate_add_node(p.config.genome_config)
h.mutate_add_node(p.config.genome_config)
h.mutate_add_node(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)
h.mutate_add_connection(p.config.genome_config)

In [14]:
print(h)

Key: 9
Fitness: 1.0968289375305176
Nodes:
	0 DefaultNodeGene(key=0, bias=0.2840055525302887, response=1.0, activation=sigmoid, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-0.22266171274789123, response=1.0, activation=sigmoid, aggregation=sum)
	4 DefaultNodeGene(key=4, bias=-1.952108050005937, response=1.0, activation=sigmoid, aggregation=sum)
	5 DefaultNodeGene(key=5, bias=-0.41877374337595313, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=-0.6640235819833576, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.6365209221839905, enabled=False)
	DefaultConnectionGene(key=(-2, 4), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 5), weight=0.7796292335753101, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.11565177142620087, enabled=False)
	DefaultConnectionGene(key=(-1, 3), weight=1.0, enabled=False)
	DefaultConnectionGene(key=(-1, 5), weight=1.0, enabled=True)
	Defaul

In [15]:
# h.add_connection(p.config.genome_config, 16, 17, 0.2, True)
# h.add_connection(p.config.genome_config, 17, 0, 0.2, True)

In [16]:
print(h)

Key: 9
Fitness: 1.0968289375305176
Nodes:
	0 DefaultNodeGene(key=0, bias=0.2840055525302887, response=1.0, activation=sigmoid, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-0.22266171274789123, response=1.0, activation=sigmoid, aggregation=sum)
	4 DefaultNodeGene(key=4, bias=-1.952108050005937, response=1.0, activation=sigmoid, aggregation=sum)
	5 DefaultNodeGene(key=5, bias=-0.41877374337595313, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=-0.6640235819833576, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.6365209221839905, enabled=False)
	DefaultConnectionGene(key=(-2, 4), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 5), weight=0.7796292335753101, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.11565177142620087, enabled=False)
	DefaultConnectionGene(key=(-1, 3), weight=1.0, enabled=False)
	DefaultConnectionGene(key=(-1, 5), weight=1.0, enabled=True)
	Defaul

In [17]:
p.config.genome_config.input_keys

[-1, -2]

In [18]:
h.nodes

{0: <neat.genes.DefaultNodeGene at 0x7fbcd0f05b80>,
 3: <neat.genes.DefaultNodeGene at 0x7fbc808fe070>,
 4: <neat.genes.DefaultNodeGene at 0x7fbc808f2be0>,
 5: <neat.genes.DefaultNodeGene at 0x7fbc808fe1c0>,
 6: <neat.genes.DefaultNodeGene at 0x7fbc808fe2b0>}

In [19]:
node_tracker = {node_id:{'depth':0, 'output_ids':[], 'input_ids':[]} for node_id in h.nodes}
for node_id in p.config.genome_config.input_keys:
    node_tracker[node_id] = {'depth':0, 'output_ids':[], 'input_ids':[]}
trace_stack = [node_id for node_id in p.config.genome_config.input_keys]

In [20]:
trace_stack

[-1, -2]

In [21]:
node_tracker

{0: {'depth': 0, 'output_ids': [], 'input_ids': []},
 3: {'depth': 0, 'output_ids': [], 'input_ids': []},
 4: {'depth': 0, 'output_ids': [], 'input_ids': []},
 5: {'depth': 0, 'output_ids': [], 'input_ids': []},
 6: {'depth': 0, 'output_ids': [], 'input_ids': []},
 -1: {'depth': 0, 'output_ids': [], 'input_ids': []},
 -2: {'depth': 0, 'output_ids': [], 'input_ids': []}}

In [22]:
for connection in h.connections:
    print(connection)
    node_tracker[connection[0]]['output_ids'].append(connection[1])
    node_tracker[connection[1]]['input_ids'].append(connection[0])

(-1, 0)
(-2, 0)
(-1, 3)
(3, 0)
(-2, 4)
(4, 0)
(-1, 5)
(5, 3)
(-1, 6)
(6, 3)
(6, 4)
(4, 5)
(-2, 5)


In [23]:
while len(trace_stack) > 0:
    trace = trace_stack[0]
    my_depth = node_tracker[trace]['depth']
    next_depth = my_depth + 1
    for output_id in node_tracker[trace]['output_ids']:
        node_tracker[output_id]['depth'] = max(node_tracker[output_id]['depth'], next_depth)
        trace_stack.append(output_id)
    del(trace_stack[0])

In [24]:
node_tracker

{0: {'depth': 5, 'output_ids': [], 'input_ids': [-1, -2, 3, 4]},
 3: {'depth': 4, 'output_ids': [0], 'input_ids': [-1, 5, 6]},
 4: {'depth': 2, 'output_ids': [0, 5], 'input_ids': [-2, 6]},
 5: {'depth': 3, 'output_ids': [3], 'input_ids': [-1, 4, -2]},
 6: {'depth': 1, 'output_ids': [3, 4], 'input_ids': [-1]},
 -1: {'depth': 0, 'output_ids': [0, 3, 5, 6], 'input_ids': []},
 -2: {'depth': 0, 'output_ids': [0, 4, 5], 'input_ids': []}}

In [25]:
for node_id, node in node_tracker.items():
    node['output_layers']=[]
    node['needs_skip'] = False
    node['id'] = node_id
    for output_id in node['output_ids']:
        node['output_layers'].append(node_tracker[output_id]['depth'])
        if node_tracker[output_id]['depth'] > (node['depth']+1):
            node['needs_skip'] = True

Create input layer definitions

In [26]:
for node_id, node in node_tracker.items():
    node['input_layers'] = []
    node['skip_layer_input'] = False
    for input_id in node['input_ids']:
        node['input_layers'].append(node_tracker[input_id]['depth'])
        if node_tracker[input_id]['depth'] < (node['depth']-1):
            node['skip_layer_input'] = True

In [27]:
node_tracker

{0: {'depth': 5,
  'output_ids': [],
  'input_ids': [-1, -2, 3, 4],
  'output_layers': [],
  'needs_skip': False,
  'id': 0,
  'input_layers': [0, 0, 4, 2],
  'skip_layer_input': True},
 3: {'depth': 4,
  'output_ids': [0],
  'input_ids': [-1, 5, 6],
  'output_layers': [5],
  'needs_skip': False,
  'id': 3,
  'input_layers': [0, 3, 1],
  'skip_layer_input': True},
 4: {'depth': 2,
  'output_ids': [0, 5],
  'input_ids': [-2, 6],
  'output_layers': [5, 3],
  'needs_skip': True,
  'id': 4,
  'input_layers': [0, 1],
  'skip_layer_input': True},
 5: {'depth': 3,
  'output_ids': [3],
  'input_ids': [-1, 4, -2],
  'output_layers': [4],
  'needs_skip': False,
  'id': 5,
  'input_layers': [0, 2, 0],
  'skip_layer_input': True},
 6: {'depth': 1,
  'output_ids': [3, 4],
  'input_ids': [-1],
  'output_layers': [4, 2],
  'needs_skip': True,
  'id': 6,
  'input_layers': [0],
  'skip_layer_input': False},
 -1: {'depth': 0,
  'output_ids': [0, 3, 5, 6],
  'input_ids': [],
  'output_layers': [5, 4, 3, 

In [28]:
layers = {}
for node_id, node in node_tracker.items():
    if not node['depth'] in layers:
        layers[node['depth']] = {
            'nodes':{node_id:node}
        }
    else:
        layers[node['depth']]['nodes'][node_id] = node
        
# Ensure all nodes have a layer index
for layer_id, layer in layers.items():
    layer_index = 0
    for node_id, node in layer['nodes'].items():
        node['layer_index'] = layer_index
        layer_index += 1
        
        
layers

{5: {'nodes': {0: {'depth': 5,
    'output_ids': [],
    'input_ids': [-1, -2, 3, 4],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 4, 2],
    'skip_layer_input': True,
    'layer_index': 0}}},
 4: {'nodes': {3: {'depth': 4,
    'output_ids': [0],
    'input_ids': [-1, 5, 6],
    'output_layers': [5],
    'needs_skip': False,
    'id': 3,
    'input_layers': [0, 3, 1],
    'skip_layer_input': True,
    'layer_index': 0}}},
 2: {'nodes': {4: {'depth': 2,
    'output_ids': [0, 5],
    'input_ids': [-2, 6],
    'output_layers': [5, 3],
    'needs_skip': True,
    'id': 4,
    'input_layers': [0, 1],
    'skip_layer_input': True,
    'layer_index': 0}}},
 3: {'nodes': {5: {'depth': 3,
    'output_ids': [3],
    'input_ids': [-1, 4, -2],
    'output_layers': [4],
    'needs_skip': False,
    'id': 5,
    'input_layers': [0, 2, 0],
    'skip_layer_input': True,
    'layer_index': 0}}},
 1: {'nodes': {6: {'depth': 1,
    'output_ids': [3, 4],
    'i

In [92]:

LAYER_TYPE_CONNECTED = "CONNECTED"
LAYER_TYPE_INPUT = "INPUT"
LAYER_TYPE_OUTPUT = "OUTPUT"
for layer_id, layer in layers.items():
    layer['is_output_layer'] = False
    layer['is_input_layer'] = False
    layer['layer_type'] = LAYER_TYPE_CONNECTED
    # If I have the output node in me, then I am an output
    if 0 in layer['nodes']:
        layer['is_output_layer'] = True
        layer['layer_type'] = LAYER_TYPE_OUTPUT

    # If I have the first input in me, then I am the input
    if -1 in layer['nodes']:
        layer['is_input_layer'] = True
        layer['layer_type'] = LAYER_TYPE_INPUT
    biases = []

    layer['input_layers'] = []
    ## Compute the shape of required inputs
    for node_id, node in layer['nodes'].items():
        for in_layer in node['input_layers']:
            if in_layer not in layer['input_layers']:
                layer['input_layers'].append(in_layer)
    layer['input_layers'].sort()
    layer['input_shape'] = sum(len(layers[jj]['nodes']) for jj in layer['input_layers'])
    layer['weights_shape'] = (layer['input_shape'], len(layer['nodes']))


    # Handle output layer "edge" case
    if layer['is_output_layer']:
        layer['out_weights'] = []
        layer['bias'] = []
        layer['in_weights'] = [[0 for __ in layers[layer_id-1]['nodes']] for _ in layer['nodes']]
    # Handle input layer "edge" case
    elif layer['is_input_layer']:
        layer['in_weights'] = []
        layer['bias'] = []
        layer['out_weights'] = [[0 for __ in layers[layer_id+1]['nodes']] for _ in layer['nodes']]
    # Handle generic case
    else:
        layer['out_weights'] = [[0 for __ in layers[layer_id+1]['nodes']] for _ in layer['nodes']]
        layer['in_weights'] = [[0 for __ in layers[layer_id-1]['nodes']] for _ in layer['nodes']]

        layer['bias'] = [h.nodes[node_id].bias for node_id, node in layer['nodes'].items()]
        # else:
            # layer['bias'] = [0 for _ in layer['nodes']]
    
    layer_index = 0          
    for node_id, node in layer['nodes'].items():
        node['layer_index'] = layer_index
        layer_index += 1

#################################################
#################################################
#################################################
        ### POSSIBLY NEED THIS
        # if node['id'] < 0:
        #     biases.append(1)
        # else:
        #     biases.append(h.nodes[node['id']].bias)
        # for output_id in node['output_ids']:
        #     if (node['id'], output_id) in h.connections:
        #         if output_id in layers[layer_id + 1]['nodes']:
        #             layer['out_weights'][node['layer_index']][layers[layer_id + 1]['nodes'][output_id]['layer_index']] = h.connections[(node['id'], output_id)].weight
        #         else:
        #             # TODO: this is a skip layer to deal with
        #             pass
        #     else:
        #         # this is not a connection that exists, so default of 0 holds
        #         pass
        # for input_id in node['input_ids']:
        #     if (input_id, node['id']) in h.connections:
        #         if input_id in layers[layer_id - 1]['nodes']:
        #             layer['in_weights'][node['layer_index']][layers[layer_id -1]['nodes'][input_id]['layer_index']] = h.connections[(input_id, node['id'])].weight
        #         else:
        #             # TODO: this is a skip layer to deal with
        #             pass
        #     else:
        #         # this is not a connection that exists, so default of 0 holds
        #         pass
        #################################################
            ### END OF POSSIBLY NEEDING THIS
    #################################################
    #################################################
    #################################################
    #################################################


    # layer['out_tensor'] = torch.tensor(layer['out_weights'])
    # layer['bias'] = torch.tensor(layer['bias'])
    # layer['out_tensor_shape'] = layer['out_tensor'].shape
    # layer['in_tensor'] = torch.tensor(layer['in_weights'])
    # layer['in_tensor_shape'] = layer['in_tensor'].shape



    # Set up current weights
    layer['input_weights'] = np.zeros(layer['weights_shape'])
    layer_offset = 0
    # Check every layer and every node for connections
    print("my layer id is %s" % (layer_id))
    for input_layer_id in layer['input_layers']:
        input_layer = layers[input_layer_id]
        for node_id, node in input_layer['nodes'].items():
            for node_output_id in node['output_ids']:
                if node_output_id in layer['nodes']:
                    node_output = layer['nodes'][node_output_id]
                    # I HAVE THIS NODE!
                    # What is it's weight?
                    connection = h.connections[(node_id, node_output_id)]
                    print(connection)

                    if not connection.enabled:
                        continue
                    connection_weight = connection.weight
                    print(connection_weight)

                    in_weight_location = layer_offset + node['layer_index']
                    out_weight_location = node_output['layer_index']
                    print('location is')
                    print((out_weight_location, in_weight_location))
                    layer['input_weights'][in_weight_location][out_weight_location] = connection_weight
        layer_offset += len(input_layer['nodes'])

my layer id is 5
DefaultConnectionGene(key=(-1, 0), weight=-0.11565177142620087, enabled=False)
DefaultConnectionGene(key=(-2, 0), weight=-0.6365209221839905, enabled=False)
DefaultConnectionGene(key=(4, 0), weight=-0.6365209221839905, enabled=True)
-0.6365209221839905
location is
(0, 2)
DefaultConnectionGene(key=(3, 0), weight=-0.11565177142620087, enabled=True)
-0.11565177142620087
location is
(0, 3)
my layer id is 4
DefaultConnectionGene(key=(-1, 3), weight=1.0, enabled=False)
DefaultConnectionGene(key=(6, 3), weight=1.0, enabled=True)
1.0
location is
(0, 2)
DefaultConnectionGene(key=(5, 3), weight=1.0, enabled=True)
1.0
location is
(0, 3)
my layer id is 2
DefaultConnectionGene(key=(-2, 4), weight=1.0, enabled=True)
1.0
location is
(0, 1)
DefaultConnectionGene(key=(6, 4), weight=1.115361001875095, enabled=True)
1.115361001875095
location is
(0, 2)
my layer id is 3
DefaultConnectionGene(key=(-1, 5), weight=1.0, enabled=True)
1.0
location is
(0, 0)
DefaultConnectionGene(key=(-2, 5), w

In [93]:
layers

{5: {'nodes': {0: {'depth': 5,
    'output_ids': [],
    'input_ids': [-1, -2, 3, 4],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 4, 2],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'out_weights': [],
  'bias': [],
  'in_weights': [[0]],
  'out_tensor': tensor([]),
  'out_tensor_shape': torch.Size([0]),
  'in_tensor': tensor([[-0.1157]]),
  'in_tensor_shape': torch.Size([1, 1]),
  'is_input_layer': False,
  'input_layers': [0, 2, 4],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'input_weights': array([[ 0.        ],
         [ 0.        ],
         [-0.63652092],
         [-0.11565177]]),
  'layer_type': 'OUTPUT'},
 4: {'nodes': {3: {'depth': 4,
    'output_ids': [0],
    'input_ids': [-1, 5, 6],
    'output_layers': [5],
    'needs_skip': False,
    'id': 3,
    'input_layers': [0, 3, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  

Add in computation of skip layers

In [41]:
layers[2]

{'nodes': {4: {'depth': 2,
   'output_ids': [0, 5],
   'input_ids': [-2, 6],
   'output_layers': [5, 3],
   'needs_skip': True,
   'id': 4,
   'input_layers': [0, 1],
   'skip_layer_input': True,
   'layer_index': 0}},
 'is_output_layer': False,
 'is_input_layer': False,
 'out_weights': [[-0.4668377906660507]],
 'in_weights': [[1.115361001875095]],
 'bias': tensor([-1.9521]),
 'out_tensor': tensor([[-0.4668]]),
 'out_tensor_shape': torch.Size([1, 1]),
 'in_tensor': tensor([[1.1154]]),
 'in_tensor_shape': torch.Size([1, 1])}

In [ ]:
layers

The question is, how do you code the operations to dynamically create the
NN? You need to be able to do the right things in the right order. You have
a list of possible operations:
* Matrix multiple
* Concatenate (for skip layers)
* Matrix addition (for biases)
* Output

The order of operations in each layer is:

1. Concatenate input tensors
2. Matrix multiply input tensors with weights
3. Add bias tensors
4. Apply activation function

I.e., `output = f(cat(inputs) + bias)` for a given activation function f

These computations can be computed back-to-front, and then executed forward

In [65]:
ACTIVATE_OPERATION = "ACTIVATE"
OUTPUT_OPERATION = "OUTPUT"
TENADD_OPERATION = "TENADD" # Tensor ADD
ADD_BIAS_OPERATION = "BIASADD"
TENMUL_OPERATION = "TENMUL"
TENCAT_OPERATION = "TENCAT"
order_of_operations = []

# for layer_id, layer in layers.items():
#     print(layer)
#     # Output for final layer
#     if layer['is_output_layer']:
#         order_of_operations.append(OUTPUT_OPERATION)
#     # Activate
#     order_of_operations.append(ACTIVATE_OPERATION)
#     # Add Bias
#     order_of_operations.append(ADD_BIAS_OPERATION)
#     # Matrix Multiply weights

    


#     print(order_of_operations)

#     break

for layer_id in range(len(layers)):
    layer = layers[layer_id]
    print(layer)

{'nodes': {-1: {'depth': 0, 'output_ids': [0, 3, 5, 6], 'input_ids': [], 'output_layers': [5, 4, 3, 1], 'needs_skip': True, 'id': -1, 'input_layers': [], 'skip_layer_input': False, 'layer_index': 0}, -2: {'depth': 0, 'output_ids': [0, 4, 5], 'input_ids': [], 'output_layers': [5, 2, 3], 'needs_skip': True, 'id': -2, 'input_layers': [], 'skip_layer_input': False, 'layer_index': 1}}, 'is_input_layer': True, 'is_output_layer': False, 'in_weights': [], 'bias': tensor([]), 'out_weights': [[1.0], [0]], 'out_tensor': tensor([[1.],
        [0.]]), 'out_tensor_shape': torch.Size([2, 1]), 'in_tensor': tensor([]), 'in_tensor_shape': torch.Size([0])}
{'nodes': {6: {'depth': 1, 'output_ids': [3, 4], 'input_ids': [-1], 'output_layers': [4, 2], 'needs_skip': True, 'id': 6, 'input_layers': [0], 'skip_layer_input': False, 'layer_index': 0}}, 'is_output_layer': False, 'is_input_layer': False, 'out_weights': [[1.115361001875095]], 'in_weights': [[1.0, 0]], 'bias': tensor([-0.6640]), 'out_tensor': tensor([

In [58]:
a = torch.randn(2,3)
b = torch.randn(3, 1)
c = torch.matmul(a, b)

In [59]:
c

tensor([[-5.9067],
        [ 0.5591]])

In [ ]:
for layer in alyers:

In [91]:
layers

{5: {'nodes': {0: {'depth': 5,
    'output_ids': [],
    'input_ids': [-1, -2, 3, 4],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 4, 2],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'out_weights': [],
  'bias': [],
  'in_weights': [[0]],
  'out_tensor': tensor([]),
  'out_tensor_shape': torch.Size([0]),
  'in_tensor': tensor([[-0.1157]]),
  'in_tensor_shape': torch.Size([1, 1]),
  'is_input_layer': False,
  'input_layers': [0, 2, 4],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'input_weights': array([[ 0.        ],
         [ 0.        ],
         [-0.63652092],
         [-0.11565177]])},
 4: {'nodes': {3: {'depth': 4,
    'output_ids': [0],
    'input_ids': [-1, 5, 6],
    'output_layers': [5],
    'needs_skip': False,
    'id': 3,
    'input_layers': [0, 3, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'out_weights': [[0]],
  'i

In [123]:
x_input = nn.Parameter(torch.tensor([0.3, 0.4], dtype=torch.float64))

In [133]:
class Net(nn.Module):
    def __init__(self, layers):
        super(Net, self).__init__()  # just run the init of parent class (nn.Module)
        self.weights = {layer_id: self._tt(layer['input_weights'].copy()) for layer_id, layer in layers.items()}
        self.biases = {layer_id: self._tt(layer['bias'].copy()) for layer_id, layer in layers.items()}
        self.layers = {layer_id: layer['layer_type'] for layer_id, layer in layers.items()}
        self.layer_inputs = {layer_id: layer['input_layers'] for layer_id, layer in layers.items()}
        self.n_layers = len(layers)

        self._outputs = None

        # x = F.relu(self.fc1(x))

        # self.fc2 = nn.Linear(512, 10)

    @staticmethod
    def _tt(mat):
        return torch.tensor(mat, requires_grad=True, dtype=torch.float64)
    def forward(self, x):
        self._outputs = {}
        for layer_id in range(self.n_layers):
            layer_input = None
            layer_type = self.layers[layer_id]
            if layer_type == LAYER_TYPE_INPUT:
                self._outputs[layer_id] = x
                continue
            if layer_type == LAYER_TYPE_CONNECTED:
                layer_input = torch.cat([self._outputs[ii] for ii in self.layer_inputs[layer_id]])
            if layer_type == LAYER_TYPE_OUTPUT:
                layer_input = torch.cat([self._outputs[ii] for ii in self.layer_inputs[layer_id]])

            print(layer_input)
            print(self.weights[layer_id])
            print(self.biases[layer_id])

            self._outputs[layer_id] = torch.relu( torch.matmul(layer_input, self.weights[layer_id]) + self.biases[layer_id] )

            if layer_type == LAYER_TYPE_OUTPUT:
                return self._outputs[layer_id]


In [134]:
net = Net(layers)

In [136]:
net._outputs

In [137]:
net.forward(x_input)

NameError: name 'outputs' is not defined

In [132]:
net.outputs

AttributeError: 'Net' object has no attribute 'outputs'

In [63]:
# I.e., `output = f(cat(inputs) + bias)` for a given activation function f

def forward(layers, x):
    layer_outputs = {}
    curr_value = x
    for ii in range(len(layers)):
        print(ii)
        layer = layers[ii]
        if layer['is_output_layer']:
            # return curr_value
            return(layer_outputs)
        input_layers = [layer_outputs[layer_id] for layer_id in layer['input_layers']]
    return(layer_outputs)
        # layer_outputs[ii] = torch.relu(
        #     torch.add(
        #         torch.matmul(
        #             torch.cat(layer['input_layers']),
        #             layer['weights']

        #         ),
        #         layer['bias']
        #     )
        # )
        # curr_value = torch.sigmoid(torch.matmul(curr_value + layers[ii]['bias'], layers[ii]['out_tensor']) )

In [64]:
forward(layers, x_input)

0


KeyError: 'input_layers'

In [ ]:
h.nodes[0].bias